In [1]:
from data import wiki
from data import corpus
import alignment

import nltk
import pandas as pd
import pickle
from tqdm import tqdm_notebook as tqdm

tqdm().pandas()

posts = wiki.load_posts()
users = wiki.load_users(posts=posts)

markers = alignment.load_markers()

In [2]:
posts.head()

,utterance_id,user,talkpage_user,conversation_root,reply_to,timestamp,timestamp_unixtime,clean_text,raw_text,tokens,pos_tags,conjunctions,articles,prepositions,adverbs,quantifiers,impersonal_pronouns,personal_pronouns,auxiliary_verbs
524288,524288,Frightner,AnonEMouse,524288,initial_post,2007-09-07 11:49:00,1.189190940E09,You should look at all of the point on the tem...,You should look at all of the point on the tem...,"[You, should, look, at, all, of, the, point, o...","[PRP, MD, VB, IN, DT, IN, DT, NN, IN, DT, NN, ...",2,7,7,5,1,1,0,3
524289,524289,Revizionist,AnonEMouse,524288,-1,2007-09-07 15:41:00,1.189204860E09,Yes I agree. The law permits usage of document...,:::Yes I agree. The law permits usage of docum...,"[Yes, I, agree, ., The, law, permits, usage, o...","[UH, PRP, VBP, ., DT, NN, VBZ, NN, IN, NNS, ,,...",4,3,6,1,0,1,0,1
1,1,Johannes003,Vensatry,1,initial_post,2011-07-15 08:38:00,1.310744280E09,"Yes, that's good. Revathy's page looked very r...","Yes, that's good. Revathy's page looked very r...","[Yes, ,, that, 's, good, ., Revathy, 's, page,...","[UH, ,, WDT, VBZ, JJ, ., NNP, POS, NN, VBD, RB...",1,1,1,3,0,3,1,0
2,2,Johannes003,Vensatry,1,1,2011-07-15 09:15:00,1.310746500E09,"Nagma'a site, at least that filmography page, ...",": Nagma'a site, at least that filmography page...","[Nagma'a, site, ,, at, least, that, filmograph...","[NNP, NN, ,, IN, JJS, IN, NN, NN, ,, VBZ, IN, ...",1,0,4,1,1,4,0,3
3,3,Johannes003,Vensatry,1,2,2011-07-15 10:33:00,1.310751180E09,I don't think there are many such official rel...,:: I don't think there are many such official ...,"[I, do, n't, think, there, are, many, such, of...","[PRP, VBP, RB, VB, EX, VBP, JJ, JJ, JJ, JJ, NN...",4,0,4,11,2,2,1,10


In [3]:
for m in markers:
    posts[m+'_bin'] = posts[m] > 0
posts['post_len'] = posts.tokens.apply(len)

In [4]:
pairs = corpus.get_reply_pairs(posts)

In [5]:
df = pairs\
    .merge(users.add_suffix('_a'), how='left', left_on='user_a', right_index=True)\
    .merge(users.add_suffix('_b'), how='left', left_on='user_b', right_index=True)

In [6]:
df.head()

,reply_to,utterance_id_a,user_a,talkpage_user_a,conversation_root_a,reply_to_a,timestamp_a,timestamp_unixtime_a,clean_text_a,raw_text_a,...,admin_ascension_a,admin_a,centrality_a,edit_count_b,gender_b,numerical_id_b,post_count_b,admin_ascension_b,admin_b,centrality_b
524291,524290,524290,AnonEMouse,AnonEMouse,524290,initial_post,2007-09-07 13:46:00,1.189197960E09,You closed [[Wikipedia:Articles for deletion\/...,You closed [[Wikipedia:Articles for deletion/J...,...,2006-11-08,True,0.869190,42028,male,226831,105.0,2005-12-24,True,0.870627
524292,524291,524291,FayssalF,AnonEMouse,524290,524290,2007-09-07 14:31:00,1.189200660E09,"Well, notability cannot be acquired in 4 days....",":Well, notability cannot be acquired in 4 days...",...,2005-12-24,True,0.870627,13150,unknown,779578,535.0,2006-11-08,True,0.869190
524293,524292,524292,AnonEMouse,AnonEMouse,524290,524291,2007-09-07 14:32:00,1.189200720E09,"No, no, I was the nominator! Just wanted to ma...",":: No, no, I was the nominator! Just wanted to...",...,2006-11-08,True,0.869190,42028,male,226831,105.0,2005-12-24,True,0.870627
262148,262147,262147,Venerock,Dana boomer,262147,initial_post,2010-10-03 18:54:00,1.286157240E09,"Hello Dana, thanks for contribute in this [[Te...","Hello Dana, thanks for contribute in this [[Te...",...,NaT,False,0.749778,33852,female,5998007,289.0,2010-08-31,True,0.873888
262146,262145,262145,Kudpung,Dana boomer,262145,initial_post,2010-10-03 10:32:00,1.286127120E09,"H Dana, totally amiss of me - belated congra...","H Dana, totally amiss of me - belated congra...",...,2011-03-02,True,0.858322,33852,female,5998007,289.0,2010-08-31,True,0.873888


In [7]:
m_freq = posts.groupby('user')[['post_len'] + list(markers)].sum()
for m in markers:
    m_freq[m + '_b_freq'] = m_freq[m]/m_freq['post_len'] 

In [8]:
m_freq

,post_len,conjunctions,articles,prepositions,adverbs,quantifiers,impersonal_pronouns,personal_pronouns,auxiliary_verbs,conjunctions_b_freq,articles_b_freq,prepositions_b_freq,adverbs_b_freq,quantifiers_b_freq,impersonal_pronouns_b_freq,personal_pronouns_b_freq,auxiliary_verbs_b_freq
user,,,,,,,,,,,,,,,,,
100%BulletProof,113,7,10,16,12,5,4,4,9,0.061947,0.088496,0.141593,0.106195,0.044248,0.035398,0.035398,0.079646
21655,140,8,8,9,11,2,6,5,3,0.057143,0.057143,0.064286,0.078571,0.014286,0.042857,0.035714,0.021429
Al Ameer son,233,17,18,26,15,6,10,10,8,0.072961,0.077253,0.111588,0.064378,0.025751,0.042918,0.042918,0.034335
AngelOfSadness,355,22,12,34,27,4,20,21,24,0.061972,0.033803,0.095775,0.076056,0.011268,0.056338,0.059155,0.067606
Anonymous Dissident,53,1,4,5,5,0,0,1,2,0.018868,0.075472,0.094340,0.094340,0.000000,0.000000,0.018868,0.037736
Backslash Forwardslash,1011,43,61,67,58,17,24,36,81,0.042532,0.060336,0.066271,0.057369,0.016815,0.023739,0.035608,0.080119
Bang bang you're dead,32,1,3,3,0,0,1,1,1,0.031250,0.093750,0.093750,0.000000,0.000000,0.031250,0.031250,0.031250
Bentheadvocate,195,11,6,16,7,2,0,8,14,0.056410,0.030769,0.082051,0.035897,0.010256,0.000000,0.041026,0.071795
CanonLawJunkie,36,0,1,2,1,0,0,1,0,0.000000,0.027778,0.055556,0.027778,0.000000,0.000000,0.027778,0.000000


In [9]:
df

,reply_to,utterance_id_a,user_a,talkpage_user_a,conversation_root_a,reply_to_a,timestamp_a,timestamp_unixtime_a,clean_text_a,raw_text_a,...,admin_ascension_a,admin_a,centrality_a,edit_count_b,gender_b,numerical_id_b,post_count_b,admin_ascension_b,admin_b,centrality_b
524291,524290,524290,AnonEMouse,AnonEMouse,524290,initial_post,2007-09-07 13:46:00,1.189197960E09,You closed [[Wikipedia:Articles for deletion\/...,You closed [[Wikipedia:Articles for deletion/J...,...,2006-11-08,True,0.869190,42028,male,226831,105.0,2005-12-24,True,0.870627
524292,524291,524291,FayssalF,AnonEMouse,524290,524290,2007-09-07 14:31:00,1.189200660E09,"Well, notability cannot be acquired in 4 days....",":Well, notability cannot be acquired in 4 days...",...,2005-12-24,True,0.870627,13150,unknown,779578,535.0,2006-11-08,True,0.869190
524293,524292,524292,AnonEMouse,AnonEMouse,524290,524291,2007-09-07 14:32:00,1.189200720E09,"No, no, I was the nominator! Just wanted to ma...",":: No, no, I was the nominator! Just wanted to...",...,2006-11-08,True,0.869190,42028,male,226831,105.0,2005-12-24,True,0.870627
262148,262147,262147,Venerock,Dana boomer,262147,initial_post,2010-10-03 18:54:00,1.286157240E09,"Hello Dana, thanks for contribute in this [[Te...","Hello Dana, thanks for contribute in this [[Te...",...,NaT,False,0.749778,33852,female,5998007,289.0,2010-08-31,True,0.873888
262146,262145,262145,Kudpung,Dana boomer,262145,initial_post,2010-10-03 10:32:00,1.286127120E09,"H Dana, totally amiss of me - belated congra...","H Dana, totally amiss of me - belated congra...",...,2011-03-02,True,0.858322,33852,female,5998007,289.0,2010-08-31,True,0.873888
262153,262152,262152,Rms125a@hotmail.com,Dana boomer,262152,initial_post,2010-10-03 20:39:00,1.286163540E09,Hi. Just writing that you closed out the [[WP:...,Hi. Just writing that you closed out the [[WP:...,...,NaT,False,0.818901,33852,female,5998007,289.0,2010-08-31,True,0.873888
10,9,9,Secret of success,Vensatry,9,initial_post,2011-07-31 11:22:00,1.312136520E09,"Well, no website is 100% perfect, you got to u...","Well, no website is 100% perfect, you got to u...",...,NaT,False,0.677806,15195,male,12023980,60.0,NaT,False,0.788716
262155,262154,262154,JCDenton2052,Dana boomer,262154,initial_post,2010-10-05 02:53:00,1.286272380E09,Could you restore Category:Second wave synthpo...,Could you restore Category:Second wave synthpo...,...,NaT,False,0.781977,232528,unknown,1004750,242.0,NaT,False,0.869732
262156,262155,262155,Courcelles,Dana boomer,262154,262154,2010-10-05 02:59:00,1.286272740E09,"There would be nothing to restore, and categor...",":There would be nothing to restore, and catego...",...,NaT,False,0.869732,4907,male,6443557,12.0,NaT,False,0.781977
524303,524302,524302,Capricornis,AnonEMouse,524302,initial_post,2007-09-10 19:41:00,1.189478460E09,If you look at the contrib history of Mr. Neut...,If you look at the contrib history of Mr. Neut...,...,NaT,False,0.749812,13150,unknown,779578,535.0,2006-11-08,True,0.869190


In [10]:
df = df.merge(m_freq, how='left', left_on='user_b', right_on = 'user')

In [11]:
df.head(4)

,reply_to,utterance_id_a,user_a,talkpage_user_a,conversation_root_a,reply_to_a,timestamp_a,timestamp_unixtime_a,clean_text_a,raw_text_a,...,personal_pronouns,auxiliary_verbs,conjunctions_b_freq,articles_b_freq,prepositions_b_freq,adverbs_b_freq,quantifiers_b_freq,impersonal_pronouns_b_freq,personal_pronouns_b_freq,auxiliary_verbs_b_freq
0,524290,524290,AnonEMouse,AnonEMouse,524290,initial_post,2007-09-07 13:46:00,1.189197960E09,You closed [[Wikipedia:Articles for deletion\/...,You closed [[Wikipedia:Articles for deletion/J...,...,177,290,0.049539,0.046377,0.083531,0.053755,0.013175,0.032938,0.046640,0.076416
1,524291,524291,FayssalF,AnonEMouse,524290,524290,2007-09-07 14:31:00,1.189200660E09,"Well, notability cannot be acquired in 4 days....",":Well, notability cannot be acquired in 4 days...",...,1677,2849,0.052228,0.051730,0.086904,0.070087,0.019707,0.038087,0.039721,0.067481
2,524292,524292,AnonEMouse,AnonEMouse,524290,524291,2007-09-07 14:32:00,1.189200720E09,"No, no, I was the nominator! Just wanted to ma...",":: No, no, I was the nominator! Just wanted to...",...,177,290,0.049539,0.046377,0.083531,0.053755,0.013175,0.032938,0.046640,0.076416
3,262147,262147,Venerock,Dana boomer,262147,initial_post,2010-10-03 18:54:00,1.286157240E09,"Hello Dana, thanks for contribute in this [[Te...","Hello Dana, thanks for contribute in this [[Te...",...,772,1475,0.060006,0.064142,0.112371,0.087454,0.020683,0.040637,0.037571,0.071783


In [12]:
list(df)

['reply_to',
 'utterance_id_a',
 'user_a',
 'talkpage_user_a',
 'conversation_root_a',
 'reply_to_a',
 'timestamp_a',
 'timestamp_unixtime_a',
 'clean_text_a',
 'raw_text_a',
 'tokens_a',
 'pos_tags_a',
 'conjunctions_a',
 'articles_a',
 'prepositions_a',
 'adverbs_a',
 'quantifiers_a',
 'impersonal_pronouns_a',
 'personal_pronouns_a',
 'auxiliary_verbs_a',
 'conjunctions_bin_a',
 'articles_bin_a',
 'prepositions_bin_a',
 'adverbs_bin_a',
 'quantifiers_bin_a',
 'impersonal_pronouns_bin_a',
 'personal_pronouns_bin_a',
 'auxiliary_verbs_bin_a',
 'post_len_a',
 'utterance_id_b',
 'user_b',
 'talkpage_user_b',
 'conversation_root_b',
 'reply_to_b',
 'timestamp_b',
 'timestamp_unixtime_b',
 'clean_text_b',
 'raw_text_b',
 'tokens_b',
 'pos_tags_b',
 'conjunctions_b',
 'articles_b',
 'prepositions_b',
 'adverbs_b',
 'quantifiers_b',
 'impersonal_pronouns_b',
 'personal_pronouns_b',
 'auxiliary_verbs_b',
 'conjunctions_bin_b',
 'articles_bin_b',
 'prepositions_bin_b',
 'adverbs_bin_b',
 'quan

In [33]:
import math
import numpy as np

def binomial(n, r):
    ''' Binomial coefficient, nCr, aka the "choose" function 
        n! / (r! * (n - r)!)
        https://stackoverflow.com/questions/26560726/python-binomial-coefficient/26561091#26561091
    '''
    assert 0 <= r <= n
    p = 1    
    for i in range(1, min(r, n - r) + 1):
        p *= n
        p //= i
        n -= 1
    return p

def binomial_prob(k,n,p):
    """
    https://en.wikipedia.org/wiki/Binomial_distribution#Probability_mass_function
    """
    assert 0 <= p <= 1
    return binomial(n,k) * math.pow(p,k) * math.pow(1-p, n-k)

def wscp(m, u):
    post_len = u['post_len_b']
    m_count_b = u[m+'_b']
    m_bin_a = 1 if u[m+'_bin_a'] else -1
    m_freq_b = u[m+'_b_freq']
    m_expected_b = post_len*m_freq_b
    if m_count_b == m_expected_b:
        return 0 # if s = 0 return early
    elif m_count_b > m_expected_b:
        s = 1
    else: # m_count_b < m_expected_b
        s = -1
    try:
        p = binomial_prob(m_count_b, post_len, m_freq_b)
        score = -s*m_bin_a*math.log(p,2)
    except OverflowError:
        return np.nan
    

In [34]:
from tqdm import tqdm_notebook as tqdm
tqdm.pandas()

In [35]:
for m in markers:
    df[m+'_coord'] = df.progress_apply(lambda x:wscp(m,x), axis=1)





  0%|          | 0/172763 [00:00<?, ?it/s]



  0%|          | 1/172763 [00:02<143:28:07,  2.99s/it]



  1%|▏         | 2175/172763 [00:03<99:09:53,  2.09s/it]



  3%|▎         | 4556/172763 [00:03<68:26:49,  1.46s/it]



  4%|▍         | 6884/172763 [00:03<47:15:01,  1.03s/it]



  5%|▌         | 9217/172763 [00:03<32:36:38,  1.39it/s]



  7%|▋         | 11533/172763 [00:03<22:30:17,  1.99it/s]



  8%|▊         | 13849/172763 [00:03<15:31:39,  2.84it/s]



  9%|▉         | 16242/172763 [00:03<10:42:22,  4.06it/s]



 11%|█         | 18570/172763 [00:03<7:23:00,  5.80it/s] 



 12%|█▏        | 20900/172763 [00:03<5:05:27,  8.29it/s]



 13%|█▎        | 23285/172763 [00:03<3:30:29, 11.84it/s]



 15%|█▍        | 25646/172763 [00:04<2:25:02, 16.90it/s]



 16%|█▌        | 28023/172763 [00:04<1:39:55, 24.14it/s]



 18%|█▊        | 30351/172763 [00:04<1:08:51, 34.47it/s]



 19%|█▉        | 32752/172763 [00:04<47:24, 49.22it/s]  



 20%|██        | 35101/172763 [00:04<32:39, 70.2

 47%|████▋     | 81293/172763 [00:07<00:04, 19556.71it/s]



 48%|████▊     | 83530/172763 [00:07<00:04, 20200.27it/s]



 50%|████▉     | 85769/172763 [00:07<00:04, 20810.50it/s]



 51%|█████     | 88046/172763 [00:07<00:03, 21360.03it/s]



 52%|█████▏    | 90435/172763 [00:07<00:03, 22060.23it/s]



 54%|█████▎    | 92771/172763 [00:07<00:03, 22433.61it/s]



 55%|█████▌    | 95072/172763 [00:07<00:03, 22591.16it/s]



 56%|█████▋    | 97372/172763 [00:07<00:03, 21853.56it/s]



 58%|█████▊    | 99591/172763 [00:07<00:03, 21679.44it/s]



 59%|█████▉    | 101783/172763 [00:08<00:03, 21627.55it/s]



 60%|██████    | 104043/172763 [00:08<00:03, 21910.21it/s]



 62%|██████▏   | 106250/172763 [00:08<00:03, 21955.33it/s]



 63%|██████▎   | 108532/172763 [00:08<00:02, 22202.46it/s]



 64%|██████▍   | 110840/172763 [00:08<00:02, 22457.92it/s]



 65%|██████▌   | 113108/172763 [00:08<00:02, 22519.58it/s]



 67%|██████▋   | 115364/172763 [00:08<00:02, 21818.61it/s]



 68%|██████▊   | 

 92%|█████████▏| 158178/172763 [00:10<00:00, 23375.67it/s]



 93%|█████████▎| 160518/172763 [00:10<00:00, 23250.39it/s]



 94%|█████████▍| 162915/172763 [00:10<00:00, 23459.72it/s]



 96%|█████████▌| 165298/172763 [00:10<00:00, 23567.34it/s]



 97%|█████████▋| 167656/172763 [00:10<00:00, 23213.73it/s]



 98%|█████████▊| 170059/172763 [00:10<00:00, 23451.02it/s]



100%|█████████▉| 172465/172763 [00:11<00:00, 23629.03it/s]



100%|██████████| 172763/172763 [00:11<00:00, 15313.15it/s]



  0%|          | 0/172763 [00:00<?, ?it/s]



  0%|          | 1/172763 [00:03<154:49:29,  3.23s/it]



  1%|          | 2085/172763 [00:03<107:04:14,  2.26s/it]



  2%|▏         | 4183/172763 [00:03<74:01:43,  1.58s/it] 



  3%|▎         | 5709/172763 [00:03<51:21:07,  1.11s/it]



  4%|▍         | 7215/172763 [00:03<35:37:23,  1.29it/s]



  5%|▌         | 8765/172763 [00:03<24:42:13,  1.84it/s]



  6%|▌         | 10768/172763 [00:03<17:04:55,  2.63it/s]



  7%|▋         | 12819/172763 [00:03<

 30%|███       | 52263/172763 [00:05<02:31, 797.56it/s]



 32%|███▏      | 54702/172763 [00:05<01:45, 1123.62it/s]



 33%|███▎      | 57062/172763 [00:05<01:13, 1570.47it/s]



 34%|███▍      | 59524/172763 [00:05<00:51, 2183.82it/s]



 36%|███▌      | 61924/172763 [00:05<00:36, 3002.62it/s]



 37%|███▋      | 64376/172763 [00:05<00:26, 4075.55it/s]



 39%|███▊      | 66835/172763 [00:05<00:19, 5435.97it/s]



 40%|████      | 69318/172763 [00:06<00:14, 7099.32it/s]



 42%|████▏     | 71792/172763 [00:06<00:11, 9030.93it/s]



 43%|████▎     | 74263/172763 [00:06<00:08, 11154.02it/s]



 44%|████▍     | 76761/172763 [00:06<00:07, 13374.60it/s]



 46%|████▌     | 79228/172763 [00:06<00:06, 15498.30it/s]



 47%|████▋     | 81693/172763 [00:06<00:05, 17372.84it/s]



 49%|████▊     | 84145/172763 [00:06<00:04, 18931.17it/s]



 50%|█████     | 86580/172763 [00:06<00:04, 20190.80it/s]



 52%|█████▏    | 89042/172763 [00:06<00:03, 21342.08it/s]



 53%|█████▎    | 91476/172763 [00:

 82%|████████▏ | 141639/172763 [00:09<00:01, 23567.33it/s]



 83%|████████▎ | 143998/172763 [00:09<00:01, 23552.03it/s]



 85%|████████▍ | 146406/172763 [00:09<00:01, 23707.65it/s]



 86%|████████▌ | 148830/172763 [00:09<00:01, 23863.74it/s]



 88%|████████▊ | 151218/172763 [00:09<00:00, 23796.32it/s]



 89%|████████▉ | 153599/172763 [00:09<00:00, 23773.49it/s]



 90%|█████████ | 155977/172763 [00:09<00:00, 23703.83it/s]



 92%|█████████▏| 158348/172763 [00:09<00:00, 23667.98it/s]



 93%|█████████▎| 160753/172763 [00:09<00:00, 23779.06it/s]



 94%|█████████▍| 163132/172763 [00:10<00:00, 23601.20it/s]



 96%|█████████▌| 165520/172763 [00:10<00:00, 23682.60it/s]



 97%|█████████▋| 167955/172763 [00:10<00:00, 23877.35it/s]



 99%|█████████▊| 170344/172763 [00:10<00:00, 23797.08it/s]



100%|█████████▉| 172756/172763 [00:10<00:00, 23890.55it/s]



100%|██████████| 172763/172763 [00:10<00:00, 16125.73it/s]



  0%|          | 0/172763 [00:00<?, ?it/s]



  0%|          | 1/17276

 20%|██        | 34816/172763 [00:04<35:11, 65.32it/s]



 22%|██▏       | 37174/172763 [00:04<24:14, 93.21it/s]



 23%|██▎       | 39505/172763 [00:04<16:42, 132.92it/s]



 24%|██▍       | 41817/172763 [00:05<11:31, 189.40it/s]



 26%|██▌       | 44107/172763 [00:05<07:57, 269.60it/s]



 27%|██▋       | 46385/172763 [00:05<05:29, 383.09it/s]



 28%|██▊       | 48624/172763 [00:05<03:48, 542.98it/s]



 29%|██▉       | 50815/172763 [00:05<02:38, 767.49it/s]



 31%|███       | 53003/172763 [00:05<01:50, 1079.71it/s]



 32%|███▏      | 55347/172763 [00:05<01:17, 1512.57it/s]



 33%|███▎      | 57648/172763 [00:05<00:54, 2101.59it/s]



 35%|███▍      | 59947/172763 [00:05<00:39, 2889.07it/s]



 36%|███▌      | 62279/172763 [00:05<00:28, 3919.10it/s]



 37%|███▋      | 64562/172763 [00:06<00:20, 5214.76it/s]



 39%|███▊      | 66844/172763 [00:06<00:15, 6746.54it/s]



 40%|███▉      | 69085/172763 [00:06<00:12, 8482.67it/s]



 41%|████▏     | 71317/172763 [00:06<00:09, 10420.

In [37]:
for m in markers:
    print(df[].mean()

0.0